# Template Matching Techniques
# Author: CuongND

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

## Polar Coordinate Transformation

In [ ]:
def convert_to_polar(img, center=None, flags=cv2.WARP_FILL_OUTLIERS):
    """Convert image to polar coordinates"""
    if center is None:
        center = (img.shape[1]//2, img.shape[0]//2)
    max_radius = min(center[0], center[1], img.shape[1]-center[0], img.shape[0]-center[1])
    return cv2.linearPolar(img, center, max_radius, flags)

## Template Matching Functions

In [ ]:
def apply_template_matching(img, template, method=cv2.TM_CCOEFF_NORMED, use_polar=False):
    """Apply template matching using specified method"""
    if use_polar:
        img = convert_to_polar(img)
        template = convert_to_polar(template)
        
    if len(img.shape) == 3:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img_gray = img
        
    if len(template.shape) == 3:
        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    else:
        template_gray = template
    
    res = cv2.matchTemplate(img_gray, template_gray, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    return {
        'result': res,
        'min_val': min_val,
        'max_val': max_val,
        'min_loc': min_loc,
        'max_loc': max_loc
    }

In [ ]:
def apply_multi_scale_template_matching(img, template, scales=[1.0], method=cv2.TM_CCOEFF_NORMED, use_polar=False, grayscale=True):
    """Apply template matching at multiple scales"""
    results = []
    for scale in scales:
        h, w = template.shape[:2]
        resized_template = cv2.resize(template, (int(w * scale), int(h * scale)))
        
        res = apply_template_matching(img, resized_template, method, use_polar)
        results.append({
            'scale': scale,
            'result': res['result'],
            'max_val': res['max_val'],
            'max_loc': res['max_loc'],
            'template_size': resized_template.shape[:2]
        })
    
    best_match = max(results, key=lambda x: x['max_val'])
    return best_match

## Image Loading and Processing

In [ ]:
def load_images(input_folder, template_path=None, use_polar=True, grayscale=True):
    """Load and process all images from folder"""
    image_paths = glob(os.path.join(input_folder, '**/*.*'), recursive=True)
    image_paths = [p for p in image_paths if p.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    
    template = None
    if template_path:
        template = cv2.imread(template_path)
        if template is None:
            print(f"Warning: Could not load template from {template_path}")
    
    results = []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            if template is not None:
                res_ccoeff = apply_template_matching(img, template, cv2.TM_CCOEFF_NORMED, use_polar)
                res_ccorr = apply_template_matching(img, template, cv2.TM_CCORR_NORMED, use_polar)
                res_sqdiff = apply_template_matching(img, template, cv2.TM_SQDIFF_NORMED, use_polar)
                
                multi_scale = apply_multi_scale_template_matching(img, template, 
                                scales=[0.8, 0.9, 1.0, 1.1, 1.2], 
                                method=cv2.TM_CCOEFF_NORMED,
                                use_polar=True)
                
                results.append({
                    'original': img_rgb,
                    'ccoeff': res_ccoeff,
                    'ccorr': res_ccorr,
                    'sqdiff': res_sqdiff,
                    'multi_scale': multi_scale,
                    'name': os.path.basename(img_path),
                    'template': cv2.cvtColor(template, cv2.COLOR_BGR2RGB) if template is not None else None
                })
            else:
                results.append({
                    'original': img_rgb,
                    'name': os.path.basename(img_path)
                })
    return results

## Results Visualization

In [ ]:
def display_results(results, num_images=5, start_index=0):
    """Display comparison results for all template matching types"""
    if not results:
        print("No images to display")
        return
    
    start_index = max(0, min(start_index, len(results)-1))
    end_index = min(start_index + num_images, len(results))
    display_images = results[start_index:end_index]
    
    plt.figure(figsize=(35, 5*len(display_images)))
    for i, result in enumerate(display_images):
        plt.subplot(len(display_images), 5, 5*i+1)
        plt.imshow(result['original'])
        plt.title(f"Original\n{result['name']}")
        plt.axis('off')
        
        if 'template' in result and result['template'] is not None:
            plt.subplot(len(display_images), 5, 5*i+2)
            plt.imshow(result['template'])
            plt.title("Template")
            plt.axis('off')
        
        if 'ccoeff' in result:
            plt.subplot(len(display_images), 5, 5*i+3)
            plt.imshow(result['original'])
            
            h, w = result['template'].shape[:2] if 'template' in result else (50, 50)
            top_left = result['ccoeff']['max_loc']
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(result['original'], top_left, bottom_right, (0,255,0), 2)
            
            plt.imshow(result['original'])
            plt.title(f"CCOEFF (score: {result['ccoeff']['max_val']:.2f})")
            plt.axis('off')
        
        if 'multi_scale' in result:
            plt.subplot(len(display_images), 5, 5*i+4)
            plt.imshow(result['original'])
            
            h, w = result['multi_scale']['template_size']
            top_left = result['multi_scale']['max_loc']
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(result['original'], top_left, bottom_right, (255,0,0), 2)
            
            plt.imshow(result['original'])
            plt.title(f"Multi-scale (score: {result['multi_scale']['max_val']:.2f})")
            plt.axis('off')
        
        if 'ccoeff' in result:
            plt.subplot(len(display_images), 5, 5*i+5)
            plt.imshow(result['ccoeff']['result'], cmap='hot')
            plt.title("Matching Heatmap")
            plt.colorbar()
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()

## Example Usage

In [ ]:
# Example usage with polar coordinate transformation
template_path = 'images/templates/bia4.png'
bia4_results = load_images('images\\bo du lieu bia so 4\\train', template_path)
display_results(bia4_results, num_images=5, start_index=5)

template_path = 'images/templates/bia7.png'
bia7_results = load_images('images\\bo du lieu bia so 7\\train', template_path)
display_results(bia7_results, num_images=5, start_index=5)